In [6]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as pyplot

df = pd.read_csv("breast-cancer.csv")

df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Drop unnecessary columns 
and map 'M' to 0 and 'B' to 1

In [7]:
df.drop(columns= ["id"], errors = "ignore")
df["diagnosis"] = df["diagnosis"].map({"M": 0, "B": 1})
X = df.drop(columns = ["diagnosis"]).values
y = df["diagnosis"].values.reshape(-1,1)
print(X.shape,y.shape)

(569, 31) (569, 1)


In [8]:
np.random.seed(42)

m = X.shape[0]
indices = np.random.permutation(m)

train_size = int(0.8 * m)
train_idx = indices[:train_size]
test_idx  = indices[train_size:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(455, 31) (114, 31)
(455, 1) (114, 1)


as it is do FS and add bias term

In [9]:
mean = X_train.mean(axis=0)
std = X_train.std(axis = 0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

X_train = np.c_[np.ones((len(X_train),1)),X_train]
X_test = np.c_[np.ones((len(X_test),1)),X_test]

logistic regression involves sigmoid function and and the x in sigmoid function is similar to the ypred used in linear regression

In [10]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, theta):
    m = len(y)
    y_hat = sigmoid(X @ theta)
    eps = 1e-8
    loss = -(1/m) * np.sum(
        y * np.log(y_hat + eps) +
        (1 - y) * np.log(1 - y_hat + eps)
    )
    return loss

def gradient_descent(X, y, lr=0.1, epochs=3000):
    m, n = X.shape
    theta = np.zeros((n, 1))
    
    for i in range(epochs):
        y_hat = sigmoid(X @ theta)
        gradient = (1/m) * X.T @ (y_hat - y)
        theta -= lr * gradient
        
        if i % 500 == 0:
            print(f"Epoch {i}, Loss: {compute_cost(X, y, theta):.4f}")
    
    return theta

# train the model
theta = gradient_descent(X_train,y_train)

Epoch 0, Loss: 0.5183
Epoch 500, Loss: 0.0594
Epoch 1000, Loss: 0.0518
Epoch 1500, Loss: 0.0482
Epoch 2000, Loss: 0.0458
Epoch 2500, Loss: 0.0441


testing the output and it's accuracy

In [11]:
def predict(X, theta, threshold=0.5):
    probs = sigmoid(X @ theta)
    return (probs >= threshold).astype(int)
y_train_pred = predict(X_train, theta)
y_test_pred  = predict(X_test, theta)

train_acc = np.mean(y_train_pred == y_train)
test_acc  = np.mean(y_test_pred == y_test)

print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)


Train Accuracy: 0.9934065934065934
Test Accuracy: 0.956140350877193
